In [1]:
from api import get_chembl_info

In [2]:
print(get_chembl_info("glutamate"))

{'ChEMBL ID': 'CHEMBL2104514', 'Name': 'ARGININE GLUTAMATE', 'Molecule Type': 'Small molecule', 'SMILES': 'N=C(N)NCCC[C@H](N)C(=O)O.N[C@@H](CCC(=O)O)C(=O)O', 'Max Phase': '3.0', 'Mechanism of Action': None, 'Therapeutic Indications': None}


In [3]:
import requests
mol = "sulphasalazine"
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{mol}/property/CanonicalSMILES,IsomericSMILES,ConnectivitySMILES/JSON"
response = requests.get(url)
print(response)

<Response [200]>


In [7]:
type(response)

requests.models.Response

In [5]:
data = response.json()
data


{'PropertyTable': {'Properties': [{'CID': 5339,
    'SMILES': 'C1=CC=NC(=C1)NS(=O)(=O)C2=CC=C(C=C2)N=NC3=CC(=C(C=C3)O)C(=O)O',
    'ConnectivitySMILES': 'C1=CC=NC(=C1)NS(=O)(=O)C2=CC=C(C=C2)N=NC3=CC(=C(C=C3)O)C(=O)O'}]}}

In [6]:
props = response.json()['PropertyTable']['Properties'][0]
for field in ["CanonicalSMILES", "IsomericSMILES", "ConnectivitySMILES"]: 
    if field in props:
        print(props[field])

C1=CC=NC(=C1)NS(=O)(=O)C2=CC=C(C=C2)N=NC3=CC(=C(C=C3)O)C(=O)O


In [1]:
print("test")

test


In [1]:
molecule_name = "sulphasalazine"
chembl_url = f"https://www.ebi.ac.uk/chembl/api/data/molecule/search.json?q={molecule_name}"
response = requests.get(chembl_url)
# response.json()
molecules = response.json().get("molecules", [])
chembl_id = molecules[0]["molecule_chembl_id"]
details_url = f"https://www.ebi.ac.uk/chembl/api/data/molecule/{chembl_id}.json"
details = requests.get(details_url).json()

#details

NameError: name 'requests' is not defined

In [12]:
molecules = response.json().get("molecules", [])

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

/Users/soham/miniconda3/envs/hf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

def load_llama3(model_id: str, dtype: torch.dtype = None):
    """
    Load a Llama 3 model from Hugging Face to the best available device:
    prefer MPS (GPU) if available, else CPU.
    dtype: optional torch.float16 or torch.bfloat16 or torch.float32
    """

    # Decide device
    device = torch.device("cpu")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Load model
    # If dtype is None, let HF choose defaults; else force dtype
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=dtype,
        device_map="auto",        # this helps place layers appropriately
        low_cpu_mem_usage=True     # helpful for large models
    )

    # Move to device if not auto
    model = model.to(device)

    return model, tokenizer, device

def generate_text(model, tokenizer, device, prompt: str, max_new_tokens: int = 128):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

if __name__ == "__main__":
    # Example: using the 8B version
    model_id = "meta-llama/Meta-Llama-3-8B"  # or a quantized model like from MLX Community
    # You might try dtype=torch.float16 or bfloat16 if supported
    model, tokenizer, device = load_llama3(model_id, dtype=torch.float32)

    prompt = "Hello, how are you today?"
    output = generate_text(model, tokenizer, device, prompt, max_new_tokens=100)
    print("Generated:", output)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
401 Client Error. (Request ID: Root=1-68d8961f-59225fe3427384713e527188;10b102fb-88a5-4e66-b6c9-27485086c0f8)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must have access to it and be authenticated to access it. Please log in.

In [9]:
# import joblib

# model_names = ['KNN', 'SVM', 'RF', 'LR', 'XGB']
# loaded_models = {name: joblib.load(f'../../output/models/{name}_model.pkl') for name in model_names }
# MLP_model = tf.keras.models.load_model('../../output/models/MLP_model.h5')
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

import pubchempy as pcp

def get_name_from_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    name = rdMolDescriptors.CalcMolFormula(mol)
    return name

smiles = "O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O"
comp = pcp.get_compounds(smiles, 'smiles')[0]
print(comp)
print("IUPAC Name:", comp.iupac_name)
print("Common Name:", comp.synonyms[0])
print(get_name_from_smiles(smiles))




Compound(5339)
IUPAC Name: 2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]diazenyl]benzoic acid
Common Name: sulfasalazine
C18H14N4O5S


In [1]:
print("test")

test
